<a href="https://colab.research.google.com/github/sishinami/usdjpy/blob/main/2001010219_0419%E9%80%B2%E6%8D%97%E5%A0%B1%E5%91%8A%E7%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import SVC

# df['res'] 増減値 の 0-60 を １ベクトルとする
# 目的変数のは predict分経過時点で現時点より 上か下かの True False を使う
def makevec(targetdf,vecsize=60,predict=30):
    maxindex = len(targetdf)
    resultvecs = []
    for i in range(0,maxindex-predict-vecsize):
        nowhajimari = targetdf.hajimari[i]
        endhajimari = targetdf.hajimari[i+predict+vecsize]
        y = (endhajimari-nowhajimari) > 0
        resultvec = [y]
        for j in range(0,vecsize):
            resultvec.append(targetdf.res[i+j])
        resultvecs.append(resultvec)
    return resultvecs
        
def getscore(model,X_test,Y_test):
    Y_pred = model.predict(testdf[xcol])

    print('---------------------------------------------------------------')
    print(model)
    print('confusion matrix = \n', confusion_matrix(y_true=Y_test, y_pred=Y_pred))
    print('accuracy = ', accuracy_score(y_true=Y_test, y_pred=Y_pred))
    print('precision = ', precision_score(y_true=Y_test, y_pred=Y_pred))
    print('recall = ', recall_score(y_true=Y_test, y_pred=Y_pred))
    print('f1 score = ', f1_score(y_true=Y_test, y_pred=Y_pred))


In [5]:

# データ取得元
# https://www.fxddtrading.com/bm/jp/resources/mt4-one-minute-data 
df = pd.read_csv('drive/MyDrive/USDJPYmin.csv'
  ,header=None
  , names=['date','time','hajimari','takane','yasune','end','dekidaka'])



In [6]:

# 小さいモデルでテスト
offset = 100*24*60 # 100日分
df2 = df[0:offset]
df2['res'] =df[1:offset+1]['hajimari'].reset_index(drop=True) - df2['hajimari'] # 1分前から増減値
target_df = pd.DataFrame(np.array(makevec(df2,60,180))) # もっと小さくできそうな気もするが後でやろう



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [7]:

# X,Y分離
xcol = []
for f in range(1,61):
    xcol.append(f)


# train test split
traindf = target_df[:int(offset*0.7)]
testdf = target_df[int(offset*0.7):]

# LogisticRegression
logreg = LogisticRegression()
model = logreg.fit( traindf[xcol], traindf[0])
getscore(model, testdf[xcol], testdf[0] )


---------------------------------------------------------------
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
confusion matrix = 
 [[13656  7313]
 [ 7094 14897]]
accuracy =  0.6646415270018622
precision =  0.6707339036470058
recall =  0.6774134873357283
f1 score =  0.6740571480283251


In [ ]:
from sklearn.utils import all_estimators


for (name, algorithm) in all_estimators(type_filter='classifier'):
  try:
    if (name == 'LinearSVC'):
      clf = algorithm(max_iter=10000)
    else:
      clf = algorithm()
    model = clf.fit(traindf[xcol], traindf[0])
    print('------------' + name + '---------------')
    getscore(model, testdf[xcol], testdf[0] )
  except:
    pass



------------AdaBoostClassifier---------------
---------------------------------------------------------------
AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=None)
confusion matrix = 
 [[11467  9502]
 [ 7619 14372]]
accuracy =  0.6014664804469274
precision =  0.6019938007874676
recall =  0.6535400845800555
f1 score =  0.6267088193611686
------------BaggingClassifier---------------
---------------------------------------------------------------
BaggingClassifier(base_estimator=None, bootstrap=True, bootstrap_features=False,
                  max_features=1.0, max_samples=1.0, n_estimators=10,
                  n_jobs=None, oob_score=False, random_state=None, verbose=0,
                  warm_start=False)
confusion matrix = 
 [[13458  7511]
 [12011  9980]]
accuracy =  0.5455772811918064
precision =  0.5705791549939969
recall =  0.45382201809831296
f1 score =  0.5055468314675041
------------BernoulliNB-------

/usr/local/lib/python3.7/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


------------DummyClassifier---------------
---------------------------------------------------------------
DummyClassifier(constant=None, random_state=None, strategy='warn')
confusion matrix = 
 [[10510 10459]
 [10962 11029]]
accuracy =  0.5013733705772812
precision =  0.5132632166790767
recall =  0.5015233504615525
f1 score =  0.5073253754686171
------------ExtraTreeClassifier---------------
---------------------------------------------------------------
ExtraTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    min_impurity_decrease=0.0, min_impurity_split=None,
                    min_samples_leaf=1, min_samples_split=2,
                    min_weight_fraction_leaf=0.0, random_state=None,
                    splitter='random')
confusion matrix = 
 [[10632 10337]
 [10456 11535]]
accuracy =  0.5159916201117318
precision =  0.5273866130212144
recall =  0.5245327634032104
f1 sc